In [25]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
import PIL.Image as img
from keras.layers import Dense, Conv2D , MaxPool2D , Dropout , Flatten
from keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
import seaborn as sns
import os
from glob import glob
import math
from imblearn import over_sampling, under_sampling
from IPython.display import display
import skimage

In [26]:
print(f"Your GPU(s) are:{tf.config.list_physical_devices('GPU')}")

Your GPU(s) are:[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [27]:
os.chdir("/mnt/c/Users/bveat/onedrive/documents/github/fluffy-disco") # set path to WSL specific -- change as necessary
data = pd.read_csv("./data/hmnist_28_28_L.csv")
metadata = pd.read_csv("./data/HAM10000_metadata.csv")

metadata.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


In [28]:
cancer_types_dict = {
    "akiec" : "Actinic Keratoses",
    "bcc" : "Basal Cell Carcinoma",
    "bkl" : "Benign Keratosis-like Lesions",
    "df" : "Dermatofibroma",
    "mel" : "Melanoma",
    "nv" : "Melanocytic Nevi",
    "vasc" : "Vascular Lesions"
}

# Create dict of img paths:
base_skin_set = os.getcwd() # Note: the file must be in the "data" folder!

image_id_path_dict = {os.path.splitext(os.path.basename(x))[0]: x
                     for x in glob(os.path.join(base_skin_set, '*', '*.jpg'))} 

In [29]:
mod_data = metadata

mod_data["Cell_Type"] = mod_data['dx'].map(cancer_types_dict.get)
mod_data["Path"] = mod_data["image_id"].map(image_id_path_dict.get)
mod_data["Label"] = pd.Categorical(mod_data['Cell_Type']).codes # Aligns data labels with .csv files in original download
mod_data.head(10)

,lesion_id,image_id,dx,dx_type,age,sex,localization,Cell_Type,Path,Label
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,Benign Keratosis-like Lesions,/mnt/c/Users/bveat/onedrive/documents/github/f...,2
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,Benign Keratosis-like Lesions,/mnt/c/Users/bveat/onedrive/documents/github/f...,2
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,Benign Keratosis-like Lesions,/mnt/c/Users/bveat/onedrive/documents/github/f...,2
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,Benign Keratosis-like Lesions,/mnt/c/Users/bveat/onedrive/documents/github/f...,2
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,Benign Keratosis-like Lesions,/mnt/c/Users/bveat/onedrive/documents/github/f...,2
5,HAM_0001466,ISIC_0027850,bkl,histo,75.0,male,ear,Benign Keratosis-like Lesions,/mnt/c/Users/bveat/onedrive/documents/github/f...,2
6,HAM_0002761,ISIC_0029176,bkl,histo,60.0,male,face,Benign Keratosis-like Lesions,/mnt/c/Users/bveat/onedrive/documents/github/f...,2
7,HAM_0002761,ISIC_0029068,bkl,histo,60.0,male,face,Benign Keratosis-like Lesions,/mnt/c/Users/bveat/onedrive/documents/github/f...,2
8,HAM_0005132,ISIC_0025837,bkl,histo,70.0,female,back,Benign Keratosis-like Lesions,/mnt/c/Users/bveat/onedrive/documents/github/f...,2
9,HAM_0005132,ISIC_0025209,bkl,histo,70.0,female,back,Benign Keratosis-like Lesions,/mnt/c/Users/bveat/onedrive/documents/github/f...,2


In [30]:
# Define Preprocessing Function

### Note to Self: Add Edge detection, Vignette detection / cropping. 

def preprocess(input_image):
  image1 = input_image
  grey = greyscale(image1)
  contrast = apply_contrast(grey)
  blurred = blur(contrast)
  threshold = skimage.filters.threshold_mean(blurred)
  mask = blurred < threshold
  image1[~mask] = 0
  return image1

def greyscale(image):
  image = skimage.color.rgb2gray(image)
  #for clr in range(image.shape[2]):
  #  image[:,:,clr]=image.mean(axis=2)
  return image

def apply_contrast(image):
  percentiles = np.percentile(image, (0.5, 99.5))
  # array([ 1., 28.])
  #image_dark = skimage.exposure.adjust_gamma(image, gamma=1.5,gain=1)
  image_bright = skimage.exposure.adjust_gamma(image, gamma=0.5,gain=1)
  scaled = skimage.exposure.rescale_intensity(image_bright,
                                    in_range=tuple(percentiles))
  return scaled

def blur(image):
  blurred_image = skimage.filters.gaussian(image, sigma=1.0)
  return blurred_image


In [31]:
Train_dataframe, Test_dataframe = train_test_split(mod_data, test_size = .3, stratify = mod_data["Label"]) # 70-30 split

display(print(np.unique(Train_dataframe["dx"])), print(np.unique(Test_dataframe["dx"])))

['akiec' 'bcc' 'bkl' 'df' 'mel' 'nv' 'vasc']
['akiec' 'bcc' 'bkl' 'df' 'mel' 'nv' 'vasc']


None

None

In [ ]:
oversample = over_sampling.RandomOverSampler()
Train_dataframe1, Train_dataframe1_y = oversample.fit_resample(Train_dataframe, Train_dataframe["Label"])
Test_dataframe1, Test_dataframe1_y = oversample.fit_resample(Test_dataframe, Test_dataframe["Label"])

In [41]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_gen = ImageDataGenerator(rescale = 1.0 / 255, horizontal_flip = True,
                              vertical_flip = True, brightness_range = [0.2,1.0], zoom_range = [0.5,1.0])

test_gen = ImageDataGenerator(rescale = 1.0 / 255)

train_data = train_gen.flow_from_dataframe(dataframe = Train_dataframe1, x_col = "Path", y_col = "dx", target_size = (128, 128), batch_size = 32,
                                class_mode = "categorical", shuffle = True, seed = 1729)

test_data = test_gen.flow_from_dataframe(dataframe = Test_dataframe1, x_col = "Path", y_col = "dx", target_size = (128, 128), batch_size = 32,
                                class_mode = "categorical", seed = 1729)

Found 32851 validated image filenames belonging to 7 classes.
Found 14084 validated image filenames belonging to 7 classes.


In [33]:
# Show images coming in small sample batch -- size 5
""" for batch in train_data:
    images = batch[0]
    labels = batch[1]
    for i in range(len(labels)):
        plt.imshow(images[i])
        plt.show()
        print(labels[i])
    break """

' for batch in train_data:\n    images = batch[0]\n    labels = batch[1]\n    for i in range(len(labels)):\n        plt.imshow(images[i])\n        plt.show()\n        print(labels[i])\n    break '

In [35]:
from keras.models import Model
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.utils import plot_model

# Instantiate Class Inception

class Inception(tf.keras.Model):
    # Channel refers to the specific output channel for each branch; please note that channels 2 & 3 must be given as tuples.
    def __init__(self, channel1, channel2, channel3, channel4):
        super().__init__() 
        self.branch1_1 = tf.keras.layers.Conv2D(channel1, 1, activation = "relu") # Note that we add a 1x1 convolution
        self.branch2_1 = tf.keras.layers.Conv2D(channel2[0], 1, activation = "relu")
        self.branch2_2 = tf.keras.layers.Conv2D(channel2[1], 3, activation = "relu", padding = "same")
        self.branch3_1 = tf.keras.layers.Conv2D(channel3[0], 1, activation = "relu")
        self.branch3_2 = tf.keras.layers.Conv2D(channel3[1], 5, activation = "relu", padding = "same")
        self.branch4_1 = tf.keras.layers.MaxPooling2D(pool_size = 3, strides = 1, padding = "same")
        self.branch4_2 = tf.keras.layers.Conv2D(channel4, kernel_size = 1, activation = "relu")
        
    def call(self, x):
        branch1 = self.branch1_1(x)
        branch2 = self.branch2_2(self.branch2_1(x))
        branch3 = self.branch3_2(self.branch3_1(x))
        branch4 = self.branch4_2(self.branch4_1(x))
        output = tf.keras.layers.Concatenate()([branch1, branch2, branch3, branch4])
        return output
            
            

In [36]:
model = models.Sequential()

# Block One
model.add(tf.keras.layers.Input(shape = (128, 128, 3)))
model.add(Conv2D(64, kernel_size = 7, strides = 2, activation = "relu", padding = "same")) # 7x7, with kernel overlap
model.add(MaxPooling2D(pool_size = 3, strides = 2, padding = "same"))
model.add(tf.keras.layers.BatchNormalization())

# Block Two
model.add(Conv2D(64, kernel_size = 1, activation = "relu"))
model.add(Conv2D(192, kernel_size = 3, activation = "relu", padding = "same")) # 3x3 kernel; triples channels
model.add(MaxPooling2D(pool_size = 3, strides = 2, padding = "same"))
model.add(tf.keras.layers.BatchNormalization())

# Block Three
model.add(Inception(64, (92, 128), (16, 32), 32))
model.add(Inception(128, (128, 192), (32, 96), 64))
model.add(MaxPooling2D(pool_size=3, strides=2, padding= "same"))
model.add(tf.keras.layers.BatchNormalization())

# Block Four
model.add(Inception(192, (96, 208), (16, 48), 64))
model.add(Inception(160, (112, 224), (24, 64), 64))
model.add(Inception(128, (128, 256), (24, 64), 64))
model.add(Inception(112, (144, 288), (32, 64), 64))
model.add(Inception(256, (160, 320), (32, 128), 128))
model.add(MaxPooling2D(3, 2, padding = "same"))
model.add(tf.keras.layers.BatchNormalization())

# Block Five
model.add(Inception(256, (160, 320), (32, 128), 128))
model.add(Inception(384, (192, 384), (48, 128), 128))
model.add((MaxPooling2D()))
model.add(tf.keras.layers.Flatten())
model.add(Dense(4096, activation = "relu"))
model.add(Dropout(.4))
model.add(Dense(7, activation = "softmax"))

# Summary & Plot
model.summary()
keras.utils.plot_model(model, show_shapes = True)

2023-08-19 11:51:18.832127: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-19 11:51:18.832420: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-19 11:51:18.832471: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-19 11:51:19.590940: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-19 11:51:19.591206: I tensorflow/compile

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 64)        9472      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 64)       256       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 64)        4160      
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 192)       110784    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 16, 16, 192)      0

In [37]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

callback = tf.keras.callbacks.ModelCheckpoint(filepath='best_model.h5',
                                              monitor='val_acc', 
                                              mode='max',
                                              verbose=1,
                                              save_best_only=True)
# Define the optimizer
optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)

# Compile the model
model.compile(optimizer = optimizer , loss = 'sparse_categorical_crossentropy', metrics=["accuracy"])

# Set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            learning_rate=0.00001)
# Set epochs
epochs = 25

# Set batch size
batch_size = 19 
# As a fun fact, I chose batch size 19 because the simulated balance model has 32851 images
# which, when divided by 19, creates 1729 batches -- the taxicab number I've been using as a 
# seed this whole time. Isn't that fun?!

In [42]:
with tf.device('/device:GPU:0'):
    model.fit(
        x = train_data, 
        validation_data = test_data, 
        batch_size = batch_size, 
        epochs = epochs, 
        callbacks = [callback, learning_rate_reduction]
    )

Epoch 1/25


2023-08-19 12:24:39.802084: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-08-19 12:24:46.241766: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:417] Loaded runtime CuDNN library: 8.1.0 but source was compiled with: 8.6.0.  CuDNN library needs to have matching major version and equal or higher minor version. If using a binary install, upgrade your CuDNN library.  If building from sources, make sure the library loaded at runtime is compatible with the version specified during compile configuration.
2023-08-19 12:24:46.243136: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at conv_ops_fused_impl.h:625 : UNIMPLEMENTED: DNN library is not found.
2023-08-19 12:24:46.243252: I tensorflow/core/common_runtime/ex

UnimplementedError: Graph execution error:

Detected at node 'sequential/conv2d/Relu' defined at (most recent call last):
    File "/home/bveatch/miniconda3/envs/tf/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/bveatch/miniconda3/envs/tf/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/bveatch/miniconda3/envs/tf/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/bveatch/miniconda3/envs/tf/lib/python3.9/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/home/bveatch/miniconda3/envs/tf/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 725, in start
      self.io_loop.start()
    File "/home/bveatch/miniconda3/envs/tf/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    Fil